In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('Housing.csv')
print(df.head())
print(f'{df.shape=}')

      price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0  13300000  7420         4          2        3      yes        no       no   
1  12250000  8960         4          4        4      yes        no       no   
2  12250000  9960         3          2        2      yes        no      yes   
3  12215000  7500         4          2        2      yes        no      yes   
4  11410000  7420         4          1        2      yes       yes      yes   

  hotwaterheating airconditioning  parking prefarea furnishingstatus  
0              no             yes        2      yes        furnished  
1              no             yes        3       no        furnished  
2              no              no        2      yes   semi-furnished  
3              no             yes        3      yes        furnished  
4              no             yes        2       no        furnished  
df.shape=(545, 13)


In [3]:
def normalization(data):
    mean = np.mean(data)
    value_range = max(data) - min(data)
    
    result = []
    for i in data:
        result.append((i-mean)/value_range)
    return result

In [4]:
norm = pd.DataFrame()
norm['price'] = normalization(df['price'])
norm['bedrooms'] = normalization(df['bedrooms'])
norm['bathrooms'] = normalization(df['bathrooms'])
norm['area'] = normalization(df['area'])
norm.tail()

,price,bedrooms,bathrooms,area
540,-0.255128,-0.193028,-0.095413,-0.147804
541,-0.259704,0.006972,-0.095413,-0.189041
542,-0.261189,-0.193028,-0.095413,-0.105192
543,-0.261189,0.006972,-0.095413,-0.153989
544,-0.261189,0.006972,-0.095413,-0.089384


напишіть функцію гіпотези лінійної регресії у векторному вигляді;

In [5]:
def hypothesis(X, w):
    return np.dot(X, w)

створіть функцію для обчислення функції втрат у векторному вигляді;

In [6]:
def loss_function(X, y, w):
    m = len(y)
    h = hypothesis(X, w)
    return (1/(2*m)) * np.sum((h - y)**2)

реалізуйте один крок градієнтного спуску;

In [7]:
def gradient_descent_step(X, y, w, learning_rate):
    m = len(y)
    h = hypothesis(X, w)
    gradient = (1/m) * np.dot(X.T, (h - y))
    w = w - learning_rate * gradient
    return w

знайдіть найкращі параметри $\vec{w}$ для датасету прогнозуючу ціну на будинок залежно від площі, кількості ванних кімнат та кількості спалень;

In [8]:
def grad_descent(X, y, w, learning_rate=0.01, num_iterations=4000):
    
    for i in range(num_iterations):
        w = gradient_descent_step(X, y, w, learning_rate)
    return w


In [9]:
X = norm[['area', 'bedrooms', 'bathrooms']]
y = norm['price']
# Додаємо стовпець одиниць для w0
X = np.c_[np.ones(len(X)), X]

# Ініціалізуємо вектор параметрів w
w = np.zeros(X.shape[1])
w = grad_descent(X, y, w)
print(loss_function(X, y, w))
print("Оптимальні параметри w:", w, "перша спроба")

w2 = grad_descent(X, y, w)
print(loss_function(X, y, w2))
print("Оптимальні параметри w:", w2)
w3 = grad_descent(X, y, w2)
print(loss_function(X, y, w3))
w3
print("Оптимальні параметри w:", w3, "остання спроба")

0.007185845135169099
Оптимальні параметри w: [4.46085574e-17 3.06738882e-01 1.66794846e-01 2.86068587e-01] перша спроба
0.006767364361337426
Оптимальні параметри w: [4.83074335e-17 4.10496168e-01 1.89359329e-01 3.45777107e-01]
0.006722393301433987
Оптимальні параметри w: [4.91895006e-17 4.49412886e-01 1.86864188e-01 3.57378323e-01] остання спроба


знайдіть ці ж параметри за допомогою аналітичного рішення;

In [10]:
w_analytical = np.linalg.inv(X.T @ X) @ X.T @ y
print("Оптимальні параметри w (аналітичне рішення):", w_analytical)

Оптимальні параметри w (аналітичне рішення): [1.81175091e-17 4.77142690e-01 1.76112569e-01 3.60012857e-01]


порівняйте отримані результати.

In [11]:
loss_gradient_descent = loss_function(X, y, w3)
loss_analytical_solution = loss_function(X, y, w_analytical)

print("Функція втрат (градієнтний спуск):", loss_gradient_descent)
print("Функція втрат (аналітичне рішення):", loss_analytical_solution)

Функція втрат (градієнтний спуск): 0.006722393301433987
Функція втрат (аналітичне рішення): 0.006713405108514904


### Scikit-learn

In [12]:
from sklearn import linear_model

In [13]:
model = linear_model.LinearRegression()

In [14]:
X = np.array([norm.area, norm.bedrooms, norm.bathrooms]).T
X.shape

(545, 3)

In [15]:
y = np.array(norm.price)
y.shape

(545,)

In [16]:
model.fit(X, y)

LinearRegression()

In [17]:
model.coef_

array([0.47714269, 0.17611257, 0.36001286])

In [18]:
model.intercept_

7.014204135081996e-17

In [19]:
print('scikit-learn:', model.coef_, '\nloss', loss_function(X, y, model.coef_))
print('my model:', w3[1:], '\nloss', loss_function(X, y, w3[1:]))
print('analytical solution:', w_analytical[1:], '\nloss', loss_function(X, y, w_analytical[1:]))

scikit-learn: [0.47714269 0.17611257 0.36001286] 
loss 0.006713405108514905
my model: [0.44941289 0.18686419 0.35737832] 
loss 0.006722393301433987
analytical solution: [0.47714269 0.17611257 0.36001286] 
loss 0.006713405108514905
